# Part 0: Get data set for plotting ###
using meteostat API (https://dev.meteostat.net/python/) and list of capital cities around the globe (capitals_id.txt)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) ## suppress annoying deprecation warnings

from datetime import datetime, timedelta
import pandas as pd
from meteostat import Stations, Daily, Hourly

## Time period 1-20th March 2024 
start = datetime(2024, 3, 1, 0, 0, 0, 0)
end = datetime(2024, 3, 20, 0, 0, 0, 0)
start_hourly = datetime(2024, 3, 1, 0, 0, 0, 0)
end_hourly = datetime(2024, 3, 21, 0, 0, 0, 0)
print('Fetch weather stations')
stations = Stations()
all_stations = stations.fetch(sample = True)

##  Select stations via capital list
capitals = pd.read_csv("capitals_id.txt", sep="\t",dtype=str) # Get a list of weather stations in capitals of the world
selected_stations = all_stations.loc[capitals.station,:].drop_duplicates() ## get rid of duplicates if the closest station is the same for mutliple capitals
selected_stations['station'] = selected_stations.index

print('Fetch daily weather data for selected stations')
## Get daily weather info
daily_station_weather_data = (Daily(selected_stations, start, end)).fetch()
daily_station_weather_data = daily_station_weather_data.drop(columns=['snow', 'wpgt', 'tsun'])


## Get hourly data and weather codes
dict_weather_codes = {
    1:'Clear', 
    2:'Fair', 
    3:'Cloudy', 
    4:'Overcast', 
    5:'Fog', 
    6:'Freezing Fog', 
    7: 'Light Rain', 
    8: 'Rain', 
    9: 'Heavy Rain', 
    10: 'Freezing Rain',
    11: 'Heavy Freezing Rain', 
    12: 'Sleet', 
    13: 'Heavy Sleet',
    14: 'Light Snowfall',
    15:	'Snowfall',
    16: 'Heavy Snowfall',
    17:	'Rain Shower',
    18:	'Heavy Rain Shower',
    19: 'Sleet Shower',
    20: 'Heavy Sleet Shower',
    21:	'Snow Shower',
    22:	'Heavy Snow Shower',
    23:	'Lightning',
    24:	'Hail',
    25:	'Thunderstorm',
    26:	'Heavy Thunderstorm',
    27:	'Storm'}

print('Fetch hourly weather data for selected stations')
hourly_station_weather_data = Hourly(selected_stations, start_hourly, end_hourly)
hourly_weather_data = hourly_station_weather_data.fetch()

hourly_weather_data = hourly_weather_data[['dwpt','rhum', 'coco']]
hourly_weather_data['coco'] = hourly_weather_data['coco'].map(dict_weather_codes)

hourly_weather_data.reset_index(inplace=True)
daily_measurements = hourly_weather_data[hourly_weather_data['time'].dt.time == pd.to_datetime('16:00:00').time()]

## Prepare merging of daily and hourly
daily_measurements.loc[:,'time'] = daily_measurements.loc[:,'time'].dt.date
daily_measurements.set_index(['station','time'], inplace=True)

## Merge data
all_weather_data= pd.merge(daily_measurements, daily_station_weather_data, on=['station','time'])
all_weather_data = pd.merge(selected_stations, all_weather_data.reset_index(), how='right' , on=['station'])
print('Save weather data')

# Save data
all_weather_data.to_csv('global_weather.csv',  date_format='%Y-%m-%d', index=False)

print(all_weather_data)